In [1]:
import tqdm
import multiprocessing
import numpy as np
import os, glob, json

# Set seed
seed = 23
rep_size = 10
np.random.seed(seed)

set_ = 'train'
path = f'/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/{set_}/'
img_name = glob.glob(path + '*.jpg')
print(img_name[:100])


def sampling_image(i, img_name, rep_size):
    sampling = {"pair":{}}
    source_img = img_name[i]
    # target_img = np.random.choice(np.delete(img_name, i), size=rep_size, replace=False)
    target_idx = np.random.choice(np.delete(list(range(0, len(img_name))), i), size=rep_size, replace=False)
    target_img = [img_name[idx] for idx in target_idx]
    
    source_occur = [0] * len(img_name)
    target_occur = [0] * len(img_name)
    for j, pair_id in enumerate(range((i*rep_size)+1, (i*rep_size)+rep_size+1)):
        sampling["pair"][f"pair{(pair_id)}"] = {"src":source_img.split('/')[-1], 
                                          "dst":target_img[j].split('/')[-1]}
        source_occur[i] += 1
        target_occur[target_idx[j]] += 1
    return {'out_dict':sampling, 'source_occur':source_occur, 'target_occur':target_occur}

if __name__ == '__main__':
    # Your list of img_name and other setup

    num_processes = multiprocessing.cpu_count()  # You can adjust this based on your available CPU cores
    with multiprocessing.Pool(num_processes) as pool:
        # results = pool.starmap(sampling_image, [(i, img_name, rep_size) for i in range(1000)])
        results = pool.starmap(sampling_image, [(i, img_name, rep_size) for i in range(len(img_name))])


['/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/22242.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/2528.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/57366.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/44977.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/41912.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/38078.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/161.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/14777.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/532.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/2127.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/50472.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/27148.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/8093.jpg', '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/train/34552.jpg', '/data/mint/DPM_Dataset/ff

In [2]:
print(len(results))
print(results[0])
print(results[1])
print(results[2])

60000
{'out_dict': {'pair': {'pair1': {'src': '22242.jpg', 'dst': '47664.jpg'}, 'pair2': {'src': '22242.jpg', 'dst': '21481.jpg'}, 'pair3': {'src': '22242.jpg', 'dst': '46538.jpg'}, 'pair4': {'src': '22242.jpg', 'dst': '26394.jpg'}, 'pair5': {'src': '22242.jpg', 'dst': '54544.jpg'}, 'pair6': {'src': '22242.jpg', 'dst': '34366.jpg'}, 'pair7': {'src': '22242.jpg', 'dst': '26070.jpg'}, 'pair8': {'src': '22242.jpg', 'dst': '4152.jpg'}, 'pair9': {'src': '22242.jpg', 'dst': '23399.jpg'}, 'pair10': {'src': '22242.jpg', 'dst': '7265.jpg'}}}, 'source_occur': [10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [3]:
sampling = {'pair':{}}
source_occur = np.array([0] * len(img_name))
target_occur = np.array([0] * len(img_name))
for i in tqdm.tqdm(range(len(results))):
    sampling['pair'].update(results[i]['out_dict']['pair'])
    source_occur += np.array(results[i]['source_occur'])
    target_occur += np.array(results[i]['target_occur'])

print("#n samples : ", len(sampling['pair'].keys()))

100%|██████████| 60000/60000 [06:33<00:00, 152.64it/s]

#n samples :  600000


# Examine sampling coverage

In [17]:
print("="*50)
print("Source occurence")
print("Min-Max occurence of source image : ", np.min(source_occur), np.max(source_occur))
print("Mean occurence of source image : ", np.mean(source_occur))
print("Std occurence of source image : ", np.std(source_occur))
print("Unique occurence of source image : ", np.unique(source_occur))
print("="*50)

print("Target occurence")
print("Min-Max occurence of target image : ", np.min(target_occur), np.max(target_occur))
print("Mean occurence of target image : ", np.mean(target_occur))
print("Std occurence of target image : ", np.std(target_occur))
print("Unique occurence of target image : ", np.unique(target_occur))
# print(np.where(target_occur > 1))
print(f"Number of occurence <>? : ", len(np.where(target_occur == 0)[0]))
print("="*50)

Source occurence
Min-Max occurence of source image :  10 10
Mean occurence of source image :  10.0
Std occurence of source image :  0.0
Unique occurence of source image :  [10]
Target occurence
Min-Max occurence of target image :  0 150
Mean occurence of target image :  10.0
Std occurence of target image :  17.09591081711257
Unique occurence of target image :  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97 100 104 106 107 108 109 111 114 119 120
 124 127 128 135 150]
Number of occurence <>? :  40639


In [ ]:
# Plot histogram of occurence
import plotly.graph_objects as go


# Create traces
fig = go.Figure()
# Increase figsize


fig.add_trace(go.Scatter(x=list(range(len(source_occur))), y=source_occur,
                    mode='lines+markers',
                    name='Source'))
fig.add_trace(go.Scatter(x=list(range(len(target_occur))), y=target_occur,
                    mode='lines+markers',
                    name='Target'))

fig.show()
